In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Opsiyonları tanımlama
opsiyonlar = Options()

# ChromeDriver'ın yolu belirtildi.
service = Service(r"C:\Users\user\Desktop\kaynak\chromedriver-win64\chromedriver.exe") 

# Görüntülerin alınacağı URL yolu 
browser = webdriver.Chrome(service=service, options=opsiyonlar)
browser.get("https://stock.adobe.com/tr/search/images?filters%5Bcontent_type%3Aphoto%5D=1&filters%5Bglobally_safe_collection%5D=1&filters%5Bcontent_type%3Aimage%5D=1&k=surrealism+&order=relevance&limit=100&search_page=1&search_type=usertyped&acp=&get_facets=0")

# Klasör oluşturma (YazLab/surrealism/surrealism_AdobeStock)
ana_klasor = "YazLab"
surrealism_klasor = os.path.join(ana_klasor, "surrealism")
surrealism_adobe_klasor = os.path.join(surrealism_klasor, "surrealism_AdobeStock")
os.makedirs(surrealism_adobe_klasor, exist_ok=True)

limit = 2000  # İndirilecek görüntü limiti
resim_sayisi = 0
resim_linkleri = set()  # Aynı görüntüleri tekrar indirmemek için set kullanıldı.

while resim_sayisi < limit:
    # Sayfayı aşağı kaydırarak daha fazla görüntü yüklenmesi sağlandı.
    browser.execute_script("window.scrollBy(0, 500);")
    time.sleep(2) 

    # Görüntülerin bağlantıları her kaydırmadan sonra tekrar bulundu.
    img_tagleri = browser.find_elements(By.TAG_NAME, "img") 

    for img_tag in img_tagleri:
        # 'src' özelliğinden görüntülerin URL'si elde edildi.
        resim_src = img_tag.get_attribute("src")
        if resim_src and resim_src.startswith("//"):
            resim_src = "https:" + resim_src  # Tam URL oluşturuldu.

        if resim_src and resim_src not in resim_linkleri:
            resim_linkleri.add(resim_src)
            resim_sayisi += 1
            print(f"{resim_sayisi}. resim bağlantısı: {resim_src}")

            # Görüntü indirme
            try:
                resim_icerik = requests.get(resim_src).content
                with open(f"{surrealism_adobe_klasor}/surrealism_ado_{resim_sayisi}.jpg", "wb") as dosya:
                    dosya.write(resim_icerik)
                print(f"surrealism_ado_{resim_sayisi}.jpg indirildi.")
            except Exception as e:
                print(f"{resim_sayisi}. resim indirilemedi. Hata: {e}")

            if resim_sayisi >= limit:
                break

    # "İleri" butonuna otomatik tıklama
    try:
        ileri_buton = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'button__icon')]//i[contains(@class, 'icon-arrow-right')]"))
        )
        # Butona tıklamak için üstteki butonun parent'ı seçildi
        parent_button = ileri_buton.find_element(By.XPATH, "./ancestor::button")
        browser.execute_script("arguments[0].click();", parent_button)  # Butona otomatik tıklama
        time.sleep(2)  # Yeni sayfa yüklenmesi için bekleme süresi
    except Exception as e:
        print("Yeni sayfa butonu bulunamadı veya tıklanamadı. Hata:", e)
        break  # Eğer butona tıklanamazsa döngüden çık

browser.quit()

1. resim bağlantısı: https://stock.adobe.com/v1/pics/all/default/f53a065feaef6edf1c0da97be741c32a21551565.stock_logo_2020_dark.svg
surrealism_ado_1.jpg indirildi.
2. resim bağlantısı: https://stock.adobe.com/v1/pics/all/default/ebf83e4990b4cd55df48a785cedf3a6f54a91d08.stock_logo_2020.svg
surrealism_ado_2.jpg indirildi.
3. resim bağlantısı: https://t4.ftcdn.net/jpg/02/29/34/13/360_F_229341388_SjCclvMliggA6QQ15ciRdHFdgKE0fO5y.jpg
surrealism_ado_3.jpg indirildi.
4. resim bağlantısı: https://t4.ftcdn.net/jpg/02/27/67/61/360_F_227676185_ux2qwS4abpACZAjTNvf9S14SPLyXiNiT.jpg
surrealism_ado_4.jpg indirildi.
5. resim bağlantısı: https://t4.ftcdn.net/jpg/04/06/03/69/360_F_406036929_uoQok72MOnBH4yqhsnJkXewHSuAnwo8g.jpg
surrealism_ado_5.jpg indirildi.
6. resim bağlantısı: https://t3.ftcdn.net/jpg/03/39/08/36/360_F_339083677_Et3zGYmpH839UyKMNBKeA0wTMGF2TXuY.jpg
surrealism_ado_6.jpg indirildi.
7. resim bağlantısı: https://t3.ftcdn.net/jpg/01/44/14/40/360_F_144144033_PUZw8gQBbo9EfMfzn50yKYRCauxtASeQ.